# ORM: One-to-Many Relationships with SQLAlchemy

https://docs.sqlalchemy.org/en/20/orm/quickstart.html 

[Watch Video Tutorial](https://www.youtube.com/watch?v=wvQJzMrKy9E)

https://github.com/jod35/OnetoMany-SQLALCHEMY2.0/blob/main/README.md


In [32]:
from dotenv import load_dotenv, find_dotenv
import os

_ :bool = load_dotenv(find_dotenv())

conn_str = os.environ.get('neon_db_con_str')

In [33]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
from sqlalchemy import ForeignKey

"""
table programs
    - id int pk
    - name str
    - years_of_study

table courses
    - id int pk
    - title
    - code
    - program_id => fk => programs.id
"""

'\ntable programs\n    - id int pk\n    - name str\n    - years_of_study\n\ntable courses\n    - id int pk\n    - title\n    - code\n    - program_id => fk => programs.id\n'

### Create Model

In [34]:
class Base(DeclarativeBase):
    pass


class Program(Base):
    __tablename__ = 'programs'
    id:Mapped[int] = mapped_column(primary_key=True)
    name:Mapped[str] = mapped_column(nullable=False)
    years_of_study:Mapped[int] = mapped_column(nullable=False)
    courses:Mapped[list['Course']] = relationship(backref='program',passive_deletes=True)

    def __repr__(self) -> str:
        return f"<Program {self.name}>"

In [35]:
class Course(Base):
    __tablename__ = 'courses'
    id:Mapped[int] = mapped_column(primary_key=True)
    title:Mapped[str] = mapped_column(nullable=False)
    code:Mapped[str] = mapped_column(nullable=False)
    program_id:Mapped[int] = mapped_column(ForeignKey('programs.id',ondelete='CASCADE'))

    def __repr__(self) -> str:
        return f"<Course {self.title}>"

### Connect and Create Engine and Session

In [36]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine.base import Engine
from sqlalchemy.orm.session import Session


engine: Engine = create_engine(
    conn_str,
    echo= True
)


Session = sessionmaker(bind=engine)

db: Session = Session()


### Create Database

In [37]:
Base.metadata.create_all(bind=engine)

2024-06-10 17:58:17,589 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-06-10 17:58:17,589 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-10 17:58:18,216 INFO sqlalchemy.engine.Engine select current_schema()
2024-06-10 17:58:18,217 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-10 17:58:18,843 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-06-10 17:58:18,844 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-06-10 17:58:19,470 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-10 17:58:19,472 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

### Populate Database

In [38]:
program1 = Program(
    name = "Bachelors in CS",
    years_of_study =3
)

print(repr(program1))

program2 = Program(
    name = "Bachelors in Business",
    years_of_study =3
)

# saving programs
db.add_all(
    [program1,program2]
)

db.commit()

<Program Bachelors in CS>
2024-06-10 17:58:26,878 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-10 17:58:26,880 INFO sqlalchemy.engine.Engine INSERT INTO programs (name, years_of_study) SELECT p0::VARCHAR, p1::INTEGER FROM (VALUES (%(name__0)s, %(years_of_study__0)s, 0), (%(name__1)s, %(years_of_study__1)s, 1)) AS imp_sen(p0, p1, sen_counter) ORDER BY sen_counter RETURNING programs.id, programs.id AS id__1
2024-06-10 17:58:26,881 INFO sqlalchemy.engine.Engine [generated in 0.00010s (insertmanyvalues) 1/1 (ordered)] {'years_of_study__0': 3, 'name__0': 'Bachelors in CS', 'years_of_study__1': 3, 'name__1': 'Bachelors in Business'}
2024-06-10 17:58:27,507 INFO sqlalchemy.engine.Engine COMMIT


In [40]:
#create course objects
course1 = Course(
    title ="Database Admnistration",
    code = "CS 107"
)


course2 = Course(
    title ="AI",
    code = "CS 104"
)


course3 = Course(
    title ="GIS",
    code = "CS 114"
)

course4 = Course(
    title ="Businnes Administration",
    code = "BS 124"
)


# adding child object to parent
program1.courses.append(course1)
program1.courses.append(course2)
program1.courses.append(course3)

program2.courses.append(course4)

db.commit()

2024-06-10 18:00:02,452 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-10 18:00:02,455 INFO sqlalchemy.engine.Engine SELECT programs.id AS programs_id, programs.name AS programs_name, programs.years_of_study AS programs_years_of_study 
FROM programs 
WHERE programs.id = %(pk_1)s
2024-06-10 18:00:02,457 INFO sqlalchemy.engine.Engine [cached since 55.71s ago] {'pk_1': 1}
2024-06-10 18:00:03,087 INFO sqlalchemy.engine.Engine SELECT courses.id AS courses_id, courses.title AS courses_title, courses.code AS courses_code, courses.program_id AS courses_program_id 
FROM courses 
WHERE %(param_1)s = courses.program_id
2024-06-10 18:00:03,088 INFO sqlalchemy.engine.Engine [cached since 55.72s ago] {'param_1': 1}
2024-06-10 18:00:03,407 INFO sqlalchemy.engine.Engine INSERT INTO courses (title, code, program_id) SELECT p0::VARCHAR, p1::VARCHAR, p2::INTEGER FROM (VALUES (%(title__0)s, %(code__0)s, %(program_id__0)s, 0), (%(title__1)s, %(code__1)s, %(program_id__1)s, 1), (%(title__2)s, %(code

### Query Database

In [41]:
myprogram1: Program = db.query(Program).filter_by(name = "Bachelors in CS").first()
print(myprogram1.name)

mycourse3: Course = db.query(Course).filter_by(title='AI').first()
print(mycourse3.title)

2024-06-10 18:00:52,951 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-10 18:00:52,954 INFO sqlalchemy.engine.Engine SELECT programs.id AS programs_id, programs.name AS programs_name, programs.years_of_study AS programs_years_of_study 
FROM programs 
WHERE programs.name = %(name_1)s 
 LIMIT %(param_1)s
2024-06-10 18:00:52,955 INFO sqlalchemy.engine.Engine [generated in 0.00143s] {'name_1': 'Bachelors in CS', 'param_1': 1}
Bachelors in CS
2024-06-10 18:00:53,583 INFO sqlalchemy.engine.Engine SELECT courses.id AS courses_id, courses.title AS courses_title, courses.code AS courses_code, courses.program_id AS courses_program_id 
FROM courses 
WHERE courses.title = %(title_1)s 
 LIMIT %(param_1)s
2024-06-10 18:00:53,584 INFO sqlalchemy.engine.Engine [generated in 0.00099s] {'title_1': 'AI', 'param_1': 1}
AI


### Delete Rows

In [42]:
db.delete(myprogram1)

db.commit()

2024-06-10 18:01:14,578 INFO sqlalchemy.engine.Engine DELETE FROM programs WHERE programs.id = %(id)s
2024-06-10 18:01:14,579 INFO sqlalchemy.engine.Engine [generated in 0.00137s] {'id': 1}
2024-06-10 18:01:14,950 INFO sqlalchemy.engine.Engine COMMIT
